In [ ]:
!pip install PyGithub

In [ ]:
!pip install pyramid

In [ ]:
!pip install telebot

In [ ]:
!pip3 uninstall telebot -y
!pip3 uninstall PyTelegramBotAPI -y
!pip3 install pyTelegramBotAPI -y
!pip3 install --upgrade pyTelegramBotAPI -y

In [ ]:
from github import Github

# First create a Github instance:
github_token='VERY-EXAMPLE-GITHUB-TOKEN'

# using an access token
g = Github(login_or_token=github_token)

# # Github Enterprise with custom hostname
# g = Github(base_url="https://ul611/api/v3", login_or_token=github_token)

# Then play with your Github objects:
for repo in g.get_user().get_repos():
    print(repo.name)

In [ ]:
import os

# get evvironment variables
AFTER = ''
BEFORE = ''
AFTER = AFTER[1:].replace('_Некоторые особенности написания кода и запуска популярных юнит-тестов проектов', ''
                         ).replace(' на Linux._', '')
BEFORE = BEFORE[1:].replace('_Некоторые особенности написания кода и запуска популярных юнит-тестов проектов', ''
                           ).replace(' на Linux._', '')

# split the string with the names of projects
AFTER = set(AFTER.split(', '))
BEFORE = set(BEFORE.split(', '))

# print added progect name
PROJECT_NAME = ', '.join(list(AFTER.difference(BEFORE)))
print(PROJECT_NAME)

In [15]:
import telebot
from threading import Timer
import requests

TOKEN = 'EXAMPLETOKEN'

bot = telebot.TeleBot(TOKEN)

chat_id = 1000000000  #example chat id, use bots to get this

@bot.message_handler(commands=['start'])
def start_message(message):
    bot.send_message(message.chat.id,"Привет ✌️ ")
bot.polling(none_stop=True)

In [14]:
requests.__version__

'2.26.0'

In [16]:
import requests

project_name = ''
action = 'updated in' if 'edited' in project_name else 'added to'


requests.get(f'https://api.telegram.org/bot{TOKEN}/sendMessage', params=dict(
   chat_id=chat_id,
   text=f'Info on tips for 42 project {project_name} was {action} repo'
))

<Response [200]>

In [ ]:
bot=telebot.Telebot(TOKEN)
@bot.message_handler(commands=['start'])
def start_message(message):
    bot.send_message(message.chat.id,"Привет ✌️ ")
bot.infinity_poling()

In [ ]:
def send_tip():
    Timer(2.0, send_tip).start()
    bot.send_message(chat_id, config.get_random_wisdom('one'))

@bot.message_handler(commands=['start'])
def welcome(message):
    
    bot.send_message(message.chat.id, "Please, be kind and grateful for people",
        parse_mode='html')
    with open('chat_id.txt', 'w') as f:
        print(message.chat.id, file=f)
    print(message.chat.id)

#Run
bot.polling(none_stop=True) # это как While(True)


from requests import get
from bs4 import BeautifulSoup
import random

def get_random_wisdom():
    
    webpage = get('https://www.oberlo.com/blog/motivational-quotes').text
    wisdoms = [tag.get_text() 
        for tag in BeautifulSoup(webpage, 'html.parser').find_all('span') 
        if tag.get_text() and tag.get_text()[0] == '“' and "–" in tag.get_text()]
    
    return random.choice(wisdoms)



get_random_wisdom()

In [ ]:
from wsgiref.simple_server import make_server
from pyramid.config import Configurator
from pyramid.view import view_config, view_defaults
from pyramid.response import Response
from github import Github

ENDPOINT = "webhook"

@view_defaults(
    route_name=ENDPOINT, renderer="json", request_method="POST"
)
class PayloadView(object):
    """
    View receiving of Github payload. By default, this view it's fired only if
    the request is json and method POST.
    """

    def __init__(self, request):
        self.request = request
        # Payload from Github, it's a dict
        self.payload = self.request.json

    @view_config(header="X-Github-Event:push")
    def payload_push(self):
        """This method is a continuation of PayloadView process, triggered if
        header HTTP-X-Github-Event type is Push"""
        print("No commits in push:", len(self.payload['commits']))
        return Response("success")

    @view_config(header="X-Github-Event:pull_request")
    def payload_pull_request(self):
        """This method is a continuation of PayloadView process, triggered if
        header HTTP-X-Github-Event type is Pull Request"""
        print("PR", self.payload['action'])
        print("No. Commits in PR:", self.payload['pull_request']['commits'])

        return Response("success")

    @view_config(header="X-Github-Event:ping")
    def payload_else(self):
        print("Pinged! Webhook created with id {}!".format(self.payload["hook"]["id"]))
        return {"status": 200}


def create_webhook():
    """ Creates a webhook for the specified repository.

    This is a programmatic approach to creating webhooks with PyGithub's API. If you wish, this can be done
    manually at your repository's page on Github in the "Settings" section. There is a option there to work with
    and configure Webhooks.
    """

    GITHUB_TOKEN = github_token
    USERNAME = "ul611"
    PASSWORD = ""
    OWNER = ""
    REPO_NAME = "Linux_features_42tests"
    EVENTS = ["push"]
    HOST = "ul611"

    config = {
        "url": f"http://{HOST}/{ENDPOINT}".format(host=HOST, endpoint=ENDPOINT),
        "content_type": "json"
    }

    g = Github(GITHUB_TOKEN)
    repo = g.get_repo(f"{USERNAME}/{REPO_NAME}".format(owner=OWNER, repo_name=REPO_NAME))
    repo.create_hook("web", config, EVENTS, active=True)


if __name__ == "__main__":
    config = Configurator()

    create_webhook()

    config.add_route(ENDPOINT, "/{}".format(ENDPOINT))
    config.scan()

    app = config.make_wsgi_app()
    server = make_server("0.0.0.0", 80, app)
    server.serve_forever()

In [ ]:
# sha -> commit on which the status check will be created
# For example, for a webhook payload
sha = data["pull_request"]["head"]["sha"]
repo.get_commit(sha=sha).create_status(
    state="pending",
    target_url="https://FooCI.com",
    description="FooCI is building",
    context="ci/FooCI"
)